# Estudio de redes recurrentes: Hopfield 

# Programa

## Librerias

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from numba import njit  # Just-in-time compilation for faster execution opcional (si no se tiene comentar la linea y decoradores @njit)

# path de los archivos
path_ptt = "./patterns/"
path_im = "./images/"

## Patrones

In [ ]:
def Print_Pattern(patterns, save = False):
    """
        Displays and optionally saves a list of patterns as images.

        Parameters:
        patterns (list of numpy arrays): A list of 1D numpy arrays representing the patterns to be displayed.
        save (bool, optional): If True, saves the images to the specified path. Default is False.

        Returns:
        None

        Notes:
        - Each pattern in the list is reshaped into a square 2D array based on the length of the first pattern.
        - The images are displayed using matplotlib's imshow function.
        - If save is True, the images are saved to the path specified by the global variable 'path_ptt' with filenames 'Pattern_i.png', where 'i' is the pattern index.
    """
    i=0
    D = int(np.sqrt(len(patterns[0])))
    for p in patterns:
        p = p.reshape(D,D)
        plt.imshow(p)
        plt.title(f"Patrón {i}")
        if save:
            plt.savefig(os.path.join(path_ptt, f"Pattern_{i}.png"))
        i+=1

        plt.show()

def Delete_Patterns(P, n): 
    """
        Deletes the n-th pattern from the list of patterns P.
    """
    return np.delete(P,n,0)

def Import_Nist(N, delete=[], D=18):
    """
        Imports and processes MNIST data.
        Parameters:
        N (int, list, "all"): Specifies which patterns to import. If an integer, the first N patterns are imported. 
                               If a list, the patterns at the specified indices are imported. 
                               If 'all', the first 1000 patterns are imported.
        delete (list, optional): List of indices of patterns to delete after import. Defaults to an empty list.
        D (int, optional): Dimension of the square submatrix to extract from the center of each pattern. Defaults to 18.
        Returns:
        numpy.ndarray: Array of processed patterns with values -1 and 1.
    """
    
    data = np.loadtxt(os.path.join(path_ptt, 'mnist_train.csv'), delimiter=',') 
    
    if type(N) == int:
        ptts = data[:N,1:]
    elif type(N) == list:
        ptts = data[N,1:]
    elif N == 'all':
        ptts = data[:1000,1:]

    Patts = []
    for i in range(len(ptts)):
        aux = np.where(ptts[i] > 0, 1, -1)
        aux = aux.reshape(28,28)
        aux = aux[14-D//2:14+D//2,14-D//2:14+D//2]
        aux = aux.reshape(D**2)
        Patts.append(aux)


    Patts = np.array(Patts)
    delete = np.sort(delete)[::-1] #ordenar de mayor a menor delete
    for i in delete:
        Patts = Delete_Patterns(Patts, i)
    return Patts

@njit
def crear_patrones(p, N):
    """
    Creates a matrix of patterns for a Hopfield network.

    Parameters:
    p (int): The number of patterns to create.
    N (int): The number of neurons.

    Returns:
    numpy.ndarray: A matrix of shape (p, N) where each element is either 1 or -1.
    """
    random_matrix = np.random.uniform(0, 1, (p, N))
    return np.where(random_matrix > 0.5, 1, -1).astype(np.float64)


def similarity(Patts): 
    """
    Calculate the similarity matrix and mean patterns.
    This function computes a symmetric similarity matrix for a given list of patterns.
    Additionally, it returns the mean of each pattern.
    Parameters:
    Patts (list of np.ndarray): A list of patterns where each pattern is represented as a numpy array.
    Returns:
    tuple: A tuple containing:
        - float: The sum of the similarity matrix elements minus the number of patterns.
        - list: A list of mean values for each pattern.
    """
    M = np.zeros((len(Patts), len(Patts)))

    for i in range(len(Patts)):
        for j in range(i, len(Patts)):
            M[i,j] = np.abs(np.dot(Patts[i], Patts[j])/(np.linalg.norm(Patts[i])*np.linalg.norm(Patts[j])))
            M[j,i] = M[i,j]
    
    med_patts = [np.mean(p) for p in Patts]
    
    return np.sum(M) - len(Patts), med_patts

## Funciones auxiliares:
Energía - Distintas formas de W - Función de actualización

In [ ]:
@njit
def Energía(W, State, theta):
    return -0.5 * np.dot(State, np.dot(W, State)) + np.dot(theta, State)

@njit
def W_matrix_Hebb(patrones):
    """
    Computes the weight matrix using the Hebbian learning rule for a given set of patterns.

    Parameters:
    patrones (numpy.ndarray): A 2D array of shape (P, N) where P is the number of patterns 
                            and N is the number of neurons in each pattern.

    Returns:
    tuple: A tuple containing:
        - W (numpy.ndarray): The weight matrix of shape (N, N) with the weights between neurons.
        - theta (numpy.ndarray): A zero vector of the same length as the number of neurons representing the threshold for each neuron.
    """

    N = patrones.shape[1]
    W = np.zeros((N,N)).astype(np.float64)

    for i in range(len(patrones)):
        W += np.outer(patrones[i], patrones[i])

    W /= N
    np.fill_diagonal(W, 0)

    theta = np.zeros(N).astype(np.float64)

    return W , theta

@njit
def W_matrix_Storkey(patrones):
    """
    Computes the weight matrix using the Storkey learning rule for a set of patterns.
    Parameters:
    patrones (numpy.ndarray): A 2D array of shape (P, N) where P is the number of patterns 
                            and N is the number of neurons in each pattern.
    Returns:
    tuple: A tuple containing:
        - W (numpy.ndarray): The weight matrix of shape (N, N) with the weights between neurons.
        - theta (numpy.ndarray): A zero vector of the same length as the number of neurons representing the threshold for each neuron.
    """

    P, N = patrones.shape
    W = np.zeros((N, N), dtype=np.float64)

    for mu in range(P):  # Para cada patrón
        pattern = patrones[mu]

        for i in range(N):
            for j in range(N):
                if i != j:
                    h_j = np.sum(W[j, :] * pattern)  # Campo local en j
                    h_i = np.sum(W[i, :] * pattern)  # Campo local en i
                    W[i, j] += (pattern[i] * pattern[j] - pattern[i] * h_j - pattern[j] * h_i) / N

    
    np.fill_diagonal(W, 0)  # Aseguramos que los elementos diagonales son 0
    theta = np.zeros(N, dtype=np.float64)
    
    return W, theta

@njit
def W_matrix_Hebb_cm(patrones):
    """
    Computes the weight matrix using the Hebbian learning rule for a given set of patterns. 
    This version uses the  Mean-field of the patterns to calculate the threshold.

    Parameters:
    patrones (numpy.ndarray): A 2D array of shape (P, N) where P is the number of patterns 
                            and N is the number of neurons in each pattern.

    Returns:
    tuple: A tuple containing:
        - W (numpy.ndarray): The weight matrix of shape (N, N) with the weights between neurons.
        - theta (numpy.ndarray): A vector of the same length as the number of neurons representing the threshold for each neuron.
    """
    N = patrones.shape[1] # cantidad de neuronas
    W = np.zeros((N,N)).astype(np.float64)

    for i in range(len(patrones)):
        W += np.outer(patrones[i], patrones[i])

    W /= N
    np.fill_diagonal(W, 0)

    theta = 0
    # sumo sobre patrones y neuronas
    for i in range(len(patrones)):
        for j in range(len(patrones[i])):
            theta += patrones[i][j]
    
    theta /= (N*len(patrones))
    theta = np.ones(N)*theta

    return W , theta

@njit
def W_matrix_hebb_matto(patrones):
    """
    Computes the weight matrix using a modification of the Hebbian learning rule proposed by German Matto for a given set of patterns.
    This version tries to fix the problem that the patterns do not have zero mean. 

    Parameters:
    patrones (numpy.ndarray): A 2D array of shape (P, N) where P is the number of patterns 
                            and N is the number of neurons in each pattern.

    Returns:
    tuple: A tuple containing:
        - W (numpy.ndarray): The weight matrix of shape (N, N) with the weights between neurons.
        - theta (numpy.ndarray): A zero vector of the same length as the number of neurons representing the threshold for each neuron.
    """
    N = patrones.shape[1]
    W = np.zeros((N,N)).astype(np.float64)

    med_patts = [np.mean(p) for p in patrones]
    med_patts = np.array(med_patts).astype(np.float64)

    for i in range(len(patrones)):
        W += np.outer(patrones[i]-med_patts[i], patrones[i]-med_patts[i])

    W /= N
    np.fill_diagonal(W, 0)

    theta = np.zeros(N).astype(np.float64)

    return W , theta

@njit
def W_matrix_hebb_theta(patrones):
    """
    Computes the weight matrix (W) and threshold vector (theta) for a Hopfield network using the Hebbian learning rule.
    This version calculates the threshold vector as the average of each neuron's activity over all patterns.

    Parameters:
    patrones (numpy.ndarray): A 2D array where each row represents a pattern. The shape of the array is (P, N),
                              where P is the number of patterns and N is the number of neurons.

    Returns:
    tuple: A tuple containing:
        - W (numpy.ndarray): The weight matrix of shape (N, N) with zero diagonal.
        - theta (numpy.ndarray): The threshold vector of shape (N,).
    """
    
    N = patrones.shape[1]
    W = np.zeros((N,N)).astype(np.float64)

    for i in range(len(patrones)):
        W += np.outer(patrones[i], patrones[i])

    W /= N
    np.fill_diagonal(W, 0)

    theta = np.zeros(N).astype(np.float64)
    for i in range(len(patrones)):
        theta -= patrones[i]
        theta = theta / len(patrones)

    return W , theta

@njit
def Update_secuential(W, s, theta):
    """
    Perform a sequential update of the state vector `s` using the weight matrix `W` and threshold vector `theta`.

    Parameters:
    W (numpy.ndarray): The weight matrix.
    s (numpy.ndarray): The initial state vector.
    theta (numpy.ndarray): The threshold vector.

    Returns:
    tuple: A tuple containing:
        - s_new (numpy.ndarray): The updated state vector.
        - E (list): A list of energy values after each update.
    """
    E = []
    s_new = np.copy(s)
    index = np.arange(len(s))

    E.append(Energía(W, s_new, theta))

    np.random.shuffle(index)
    for j in index:
        if (np.dot(W[j], s_new) - theta[j] <0):
            s_new[j] = -1
        if (np.dot(W[j], s_new) - theta[j] >0):
            s_new[j] = 1
        else:   s_new[j] = s_new[j] # linea que no hace falta
        # if np.dot(W[j], s_new) - theta[j] == 0: s_new[j] = s_new[j]
        # else: s_new[i] = np.sign(np.dot(W[i], s_new) - theta)
        E.append(Energía(W, s_new, theta))

    return s_new, E


## Clase principal de Red Neuronal

In [ ]:
class Hopfield_NN:
    def __init__(self, N, type):
        """
        Parameters:
        N (int): The number of neurons in the network.
        type (str): The type of the network (e.g., 'binary', 'continuous').

        Attributes:
        N (int): The number of neurons in the network.
        State (np.ndarray): The state of each neuron, initialized randomly to -1 or 1.
        theta (np.ndarray): The threshold for each neuron, initialized to zeros.
        W (np.ndarray): The weight matrix of the network, initialized to zeros.
        type (str): The type Computes the weight - 'hebb', 'storkey', 'hebb_cm', 'hebb_matto', 'hebb_theta'
        """
        self.N = N
        self.State = np.random.choice([-1,1], N).astype(np.float64)
        self.theta = np.zeros(N).astype(np.float64)
        self.W = np.zeros((N,N)).astype(np.float64)
        self.type = type

    
    def Save_pattern(self, patrones):
        """
        Save the pattern into the weight matrix and threshold vector based on the specified type.

        Parameters:
        patrones (list or array-like): The patterns to be saved.
        """
        if self.type == 'hebb': self.W, self.theta = W_matrix_Hebb(patrones)
        if self.type == 'storkey': self.W, self.theta = W_matrix_Storkey(patrones)
        if self.type == 'hebb_cm': self.W, self.theta = W_matrix_Hebb_cm(patrones)
        if self.type == 'hebb_matto': self.W, self.theta = W_matrix_hebb_matto(patrones)
        if self.type == 'hebb_theta': self.W, self.theta = W_matrix_hebb_theta(patrones)
        
    
    def Update_epoch(self, print_state = False):
        """
        Updates the state of the network for one epoch using sequential update rule.

        Parameters:
        print_state (bool): If True, the function will print the state and energy plot. Default is False.

        Returns:
        E (list): The energy of the network at each iteration within the epoch
        """
        self.State , E = Update_secuential(self.W, self.State, self.theta)
        if print_state:
            fig, axes = plt.subplots(1, 2, figsize=(10, 5))
            plt.sca(axes[0])
            plt.imshow(self.State.reshape(int(np.sqrt(self.N)), int(np.sqrt(self.N))))
            plt.sca(axes[1])
            plt.plot(E)
            plt.show()
        return E
    
    
    def Energy(self):
        """
        Returns the energy of the network.
        """

        return Energía(self.W, self.State, self.theta)
    
    def Print(self):
        """
        Prints the state of the network as an image.
        """

        plt.imshow(self.State.reshape(int(np.sqrt(self.N)),int(np.sqrt(self.N))))

    def Noise(self, p):
        """
        Adds noise to the state of the network.

        Parameters:
        p (float): The percentage of neurons to change.

        Returns:
        None
        """

        #cambio p porciento de los valores
        index = np.random.choice(np.arange(self.N), int(p*self.N))
        self.State[index] = -self.State[index]
        
    def Set_state(self, state):
        """
        Sets the state of the network to the specified state.

        Parameters:
        state (np.ndarray): The state to set the network to.
        """
        
        self.State = state.astype(np.float64)

# Estudios

## Evolucion de E
Dada una red de 32*32 se estudia la evolucion de la energia en funcion de las iteraciones de una epoca. Se guardo un patron generado aleatorio y se parte de otro patron aleatorio

In [ ]:
lado = 32
NN = Hopfield_NN(lado**2, 'hebb')

fig , ax = plt.subplots(1, 2, figsize=(10, 5))
plt.sca(ax[0])
NN.Print()
plt.title("Estado inicial")

Patts = crear_patrones(1, lado**2)
plt.sca(ax[1])
Print_Pattern(Patts)
# plt.savefig("Pa.png")

NN.Save_pattern(Patts)
E = NN.Update_epoch(print_state = True)

dd = np.abs(np.array(NN.State) - np.array(Patts))
print(np.sum(dd/2))

fig = plt.figure(figsize=(5, 5))
plt.plot(E)
plt.xlabel("Neurona", fontsize=15)
plt.ylabel("Energía", fontsize=15)
plt.yticks([0,-250, -500], fontsize=15)
plt.xticks([0,512, 1024],fontsize=15)
# plt.savefig(os.path.join(path_im, "Energia.png"), dpi=300, bbox_inches='tight')

# Capacidad
Se estudia la capacidad de la red. Observando la cantidad de iteraciones para converger y el numero de errores en la recuperación

In [ ]:
N = np.array([500]); N = np.sqrt(N).astype(int) # cantidad de neuronas
alpha = np.array([0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20]) # P/N
p_noise = 0.20 # porcentaje de ruido

for n in N:  # itero sobre tamaño de la red
    Iteraciones = [] # n de it vs alpna
    Diferencia = [] # diferencia entre el patron y el estado final

    for a in alpha: # itero sobre cantidad de patrones

        print(f"N = {n**2}, p_noise = {p_noise}, alpha = {a}, {int(n**2*a)} patrones")
        Patts = crear_patrones(int(a*n**2), n**2)
        NN = Hopfield_NN(n**2, 'hebb')
        NN.Save_pattern(Patts)
        n_it = [] # cantidad de iteraciones
        n_dif = [] # diferencia entre el patron y el estado final

        for i in range(len(Patts)): # itero sobre los patrones
            NN.Set_state(Patts[i])
            NN.Noise(p_noise)
            
            it = 0
            while 1:
                it += 1
                E = NN.Update_epoch(print_state=0)
                if E[0] == E[-1]:
                    dd = np.sum(np.abs(np.array(NN.State) - np.array(Patts[i]))/2)
                    if dd >  (n**2)/2 : n_dif.append(n**2-dd) # si la diferencia es mayor a la mitad de la red, tomo el complemento 
                    else: n_dif.append(dd)
                    n_it.append(it-1)
                    break
                if it > 100: 
                    print("No converge")

                    dd = np.sum(np.abs(np.array(NN.State) - np.array(Patts[i]))/2)
                    if dd >  (n**2)/2 : n_dif.append(n**2-dd) # si la diferencia es mayor a la mitad de la red, tomo el complemento
                    else: n_dif.append(dd)
                    n_it.append(it-1)
                    break
        
        Iteraciones.append(np.mean(n_it))
        Diferencia.append(np.mean(n_dif))


fig, axes = plt.subplots(1, 2, figsize=(10, 5))
plt.sca(axes[0])
plt.plot(alpha, Iteraciones, '-o')
plt.xlabel(r'$\alpha$', fontsize=15)
plt.ylabel("Épocas", fontsize=15)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.vlines(0.14, np.min(Iteraciones), np.max(Iteraciones), color='red', linestyle='--', label=r"$\alpha_c = 0.14$")
plt.legend(fontsize=15, loc='lower right')

plt.sca(axes[1])
plt.plot(alpha, 100*np.array(Diferencia)/n**2, '-o')
plt.vlines(0.14, np.min(100*np.array(Diferencia)/n**2), np.max(100*np.array(Diferencia)/n**2), color='red', linestyle='--', label=r"$\alpha_c = 0.14$")
plt.legend(fontsize=15, loc='lower right')
plt.xlabel(r"$\alpha$", fontsize=15)
plt.ylabel("Diferencia[%]", fontsize=15)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)


## Ejemplo Práctico con patrones M_nist
para redes de 18*18, con regla de Hebb o variantes, los patrones utilizados minimizando la media y similaridad fueron: [0,1,5,12,20,25,52,85]
para redes de 18*18, con regla de Storkey, los patrones utilizados fueron: [0,1,12,20,25,52,2,3,4,6,7,8,9,13,15,16,58,60,27]

In [ ]:
Patts = Import_Nist([0,1,12,20,25,52,2,3,4,6,7,8,9,13,15,16,58,60,27], D=18)#; Print_Pattern(Patts)
NN = Hopfield_NN(18**2, 'storkey')
NN.Save_pattern(Patts)

In [ ]:
DIF = []
IT = []
for i in range(len(Patts)):
    NN.Set_state(Patts[i])
    # figura de 1x2 con el patron y noiseado
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    plt.sca(axes[0])
    NN.Print()
    plt.title(f"Patrón {i}")
    plt.sca(axes[1])
    NN.Noise(0.20)
    NN.Print()
    plt.title(f"Patrón {i} con ruido")
    plt.show()

    it = 0
    while 1:
        it += 1
        E = NN.Update_epoch(print_state = False)
        if E[0] == E[-1]:
            break
        if it > 100:
            print("No converge")
            break
    
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    plt.sca(axes[0])
    NN.Print()
    plt.title(f"Patrón {i} final, {it} iteraciones")
    IT.append(it)
    plt.sca(axes[1])
    plt.plot(E)
    diff = np.sum(np.abs(np.array(NN.State) - np.array(Patts[i]))/2)
    plt.title(f"Diferencia = {diff}")
    DIF.append(diff)